## Passo 1: Criar um ambiente virtual
##### Deve criar o ambiente na pasta principal onde será feito o projeto.
Exemplo usando o venv no windows

In [ ]:
python -m venv <nome> #pra windows

Para linux, ao chamar "python" na linha de comando, pode estar se referindo à versões 2.x. Por isso, para criar o ambiente, utiliza-se:

In [ ]:
python3 -m venv <nome> #Pra linux

## Passo 2: Ativar o ambiente

In [ ]:
. nome/Scripts/activate #pra windows

In [ ]:
source <nome>/bin/activate #pra linux

## Passo 3: instalar o django

In [ ]:
pip install Django

# para abrir o vsCode com o ambiente ativo

. code

## Passo 4: Iniciar o projeto Django

In [ ]:
django-admin startproject <nome_projeto> .

## Componentes principais que são criados:
**wsgi.py**: O arquivo wsgi.py é um componente fundamental em um projeto Django quando se trata de servir a aplicação web usando servidores web compatíveis com WSGI (Web Server Gateway Interface). Quando você implanta seu projeto Django em um servidor web, como o Apache HTTP Server com mod_wsgi ou o uWSGI, o servidor web usa o wsgi.py para interagir com a aplicação Django.
## 
**urls.py**: serve pra dizer ao navegador qual página ele vai acessar. Funciona como uma rota
##
**settings.py**: tem toda a configuração do Django

## Passo 5: Criar migrações do banco de dados

In [ ]:
python manage.py migrate

Esse comando verifica no settings.py o banco de dados que será utilizado (Por padrão é o sqlite3) e cria um banco de dados inicial.

## Passo 6: Rodar o servidor

In [ ]:
python manage.py runserver

## Passo 7: Criar um app principal que vai gerenciar o projeto

In [ ]:
python manage.py startapp <nome_app>

## componentes Principais criados:

**admin**:
##
**models**: forma que o Django dá de manipular o banco de dados. (Exemplo: para criar uma tabela, cria o prório modelo no models). está interligado com a pasta migrations, pois quando algo for alterado, as migrações irão para o banco dados.
##
**views**:

## Passo 8: adicionar o app criado no arquivo "Settings.py"

Em INSTALLED_APPS (linha 33), adicionar o nome do app criado. Assim, o projeto vai enxergar o app como uttilizável.

## Passo 8: Criar uma classe no models.py

Essa classe será usada para criar o tópico que o usuário irá registrar no app, para inserir as anotações. A classe deve herdar de models.Model. Exemplo:

In [ ]:
from django.db import models

class Topic(models.Model): # a classe se transforma em uma tabela no banco de dados
    text = models.CharField(max_length=200) #crio um campo text, do tipo CharField (de tamanho limitado - normalmente pra títulos), com tamanho máximo de 200 caracteres
    date_added = models.DateTimeField(auto_now_add=True) # vai adicionar a data e hora de forma automática em que for registrado alguma coisa
    
    def __str__(self): #Método pra organizar os tópicos, retornando uma representação em string do modelo criado
       return self.text

Após criar a classe, deve-se utilizar o makemigrations para as alterações irem pro banco de dados

In [ ]:
# Este comando é usado para criar arquivos de migração com base em alterações feitas nos modelos do seu aplicativo
python manage.py makemigrations <nome_app>

# completar a migração e faz a tabela aparecer no banco
python manage.py migrate

Fazendo isso, no banco de dados aparecerá uma nova tabela de nome <nome>_app_Topic. Nela é atribuida automaticamente um id, o text e o date_added

## Passo 9: Configurar o painel administrativo

In [ ]:
python manage.py createsuperuser #criar o login do super usuário

Username (leave blank to use 'girle'): admin
#####
Email address: (não precisa)
#####
Password: 123
#####
Password (again): 123
######
This password is too short. It must contain at least 8 characters.
This password is too common.
This password is entirely numeric.
####
Bypass password validation and create user anyway? [y/N]: y
####
Superuser created successfully.

## Passo 10: Importar a tabela criada anteriormente pro admin.py

Deve importar todos os modelos que queremos que apareça no painel administrativo, se não não vai aparecer lá.

In [ ]:
from django.contrib import admin
from nome_app.models import Topic

# registrando a tabela
admin.site.register(Topic)

# Passo 10: Criando um novo modelo que traga o texto relacionado ao modelo criado anteriormente

Obs.: Para cada Topic, posso ter várias anotações. É uma relação de um para muitos.
##
Para correlacionar, utiliza o conceito de chave estrangeira (ForeignKey), que diz que pra cada topico de texto, tem um título no banco de dados relacionado, que eu posso escolher.

In [ ]:
class Entry(models.Model):
    topic = models.ForeignKey(Topic, on_delete=models.CASCADE) #está mostrando que a chave estrangeira está vindo de topic
    #usando o cascade, significa que quando eu apagar o título, todos os textos associados serão apagados também
    text = models.TextField() #campo de texto com tamanho ilimitado
    date_added = models.DateTimeField(auto_now_add=True)
    
    class Meta: #pra quando o Django for usar o plural de entry 
        verbose_name_plural = 'entries'
        
    def __str__(self):
        return self.text[:50]+ '...' #retorna os primeiros 50 caracteres + reticências

Argumentos possíveis pro on_delete: https://docs.djangoproject.com/en/2.2/ref/models/fields/#arguments


!!! Lembrar de atualizar no admin.py!!!

In [ ]:
from django.contrib import admin
from Petra_App.models import Topic, Entry

# registrando a tabela
admin.site.register(Topic)
admin.site.register(Entry)

## Passo 11: Criar HTML e a rota para ele

Fazer com que o Django leve para uma página inicial com nosso próprio html. Para isso, importamos o caminho no urls.py do Projeto.

In [ ]:
from django.contrib import admin
from django.urls import path, include # importando também o include, pra pegar do app

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", include('Petra_App.urls')), #quando não digitar nada depois do endereço, ele deve ir pro arquivo urls
]


Em seguida, criamos um arquivo urls no APP, parecido com o do projeto:

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path("", views.index), # quando não tiver nada na url, chama a função index do views
]


Posteriormente, precisamos configurar o views para reconhecer a página html

In [ ]:
from django.shortcuts import render

def index(request):
    '''Página principal'''
    return render(request, 'Petra_App/index.html') #vai pegar tudo que está sendo trabalhado na função e vai mandar pra página html

Por fim, devemos criar uma pasta chamada templates, no app, e dentro dela uma pasta com o nome do app, que vai carregar os arquivos html.

### **Observação**: criando um arquivo com coisas em comum em todas as páginas

No base.html, podemos colocar coisas em comum, como menus cabeçalhos e rodapés. Semelhante a POO, o base é o arquivo pai e os outros erdam dele. Exemplo: criando um link que leva pra página principal:


In [ ]:
<a href="{% url 'index' %}"><h1>Home</h1></a> <!---Criando um link que me manda pro index-->

{% block content %}
<!---Aqui fica o conteúdo da página filha-->
{% endblock content%}

<footer>teste</footer> 

tudo que estiver fora do block, pode ser extendido pra todas as páginas

In [ ]:
#no index ficará:

{% extends "Petra_App/base.html" %}

{% block content %}
    <h1>Testando</h1>
{% endblock content%}

fazemos também uma modificação no urls do app, adicionando o nome da url

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path("", views.index, name='index'), # quando não tiver nada na url, chama a função index do views
]
# se for outra view, colocamos o name dela

## Passo 12: Importando o conteúdo do banco de dados pro html

Para isso, precisamos adicionar um novo método no views para acessar o banco de dados

In [ ]:
from .models import Topic #importar topic 

def topic(request):
    topics = Topic.objects.order_by('date_added') #estou pegando o objeto que está em topics e estou ordenando pela data
    context = {'topics': topics}
    return render(request, 'Petra_App/topics.html', context) # passando a variável topic como contexto para o html

Adicionamos a rota em urls (do app)

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path("", views.index, name='index'), # quando não tiver nada na url, chama a função index do views
    path("topics", views.topic, name='topics'),
]

adicionando os tópicos no html

In [ ]:
{% extends "Petra_App/base.html" %}

{% block content %}
    <ul>
        {% for topic in topics %}
        <li>
            {{topic}}
        </li>
        {% empty %}
            <li>Não há tópicos</li>
        {% endfor %}
    </ul>
{% endblock content%}

***Observação***: Se quiser colocar um tópico específico na tela:

No urls

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path("", views.index, name='index'), # quando não tiver nada na url, chama a função index do views
    path("topics", views.topics, name='topics'),
    path("topics/<topic_id>/", views.topic, name='topic'), # para pegar um tépico pelo id
]


No views:

In [ ]:
def topic(request, topic_id): #mostra um único assunto e todas as suas entradas
    topic = Topic.objects.get(id = topic_id) # pega o tópico pelo id
    entries = topic.entry_set.order_by('-date_added') #pega da tabela de entrada do mais recente pro mais antigo
    context = {'topic': topic, 'entries': entries}
    return request(request, 'Petra_App/topic.html', context)


no html topoic.html

In [ ]:
{% extends "Petra_App/base.html" %}

{% block content %}
<h1>Tópicos</h1>
    <h3>
        Tópico: {{ topic }}
    </h3>
    <p>Entradas: </p>
    <ul>
        {%for entry in entries %}
        <li>
            <p>{{entry.date_added}}</p> <!---pega a data e formatando a data-->
            <p>{{entry.text | linebreaks}}</p> <!---pega o texto e dá quebra de linhas no texto-->
        </li>
        {%endfor%}
    </ul>
{% endblock content%}